In [6]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import sys
import os

path_to_append = os.path.abspath(os.path.join(os.pardir, 'proteomics_specialist'))
sys.path.append(path_to_append)

# Now import your module
import documentation_agent as doc_agent 

In [13]:
# !pip install -q -U google-generativeai
# !pip install gradio

In [10]:
import configparser
import google.generativeai as genai

config = configparser.ConfigParser()
config.read("../secrets.ini")

api_key = config["DEFAULT"]["API_KEY"]

genai.configure(api_key=api_key)

In [12]:
# Test if the API Key is working

# model = genai.GenerativeModel("gemini-1.5-flash")  # Use the Gemini Pro model

# response = model.generate_content("Hello, how are you?")  # Send a prompt

# print(response.text)  # Print the model's response

In [15]:
# few_shot_example_folder = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/"

# # Define the few shot example and domain knowldge
# video_file_name = os.path.join(few_shot_example_folder + "tims_calibration.mov")
# video_example = doc_agent.upload_video_and_wait(
#     video_path = video_file_name
# )

# path_example = os.path.join(few_shot_example_folder + "tims_calibration.md")
# file_example = doc_agent.read_and_encode_document(path_example)

path_glossary = os.path.abspath(os.path.join(os.pardir, 'data', 'glossary.md'))
glossary = doc_agent.read_and_encode_document(path_glossary)

In [16]:
# Define the prompts
prompt_1 = "Example Input video: "
prompt_2 = "Example Output protocol:"
prompt_3 = "Glossary:"
prompt_4 =  """
You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry. You are known for your methodological rigorand clear scientific writing, particularly in the style of Nature Protocols.
Your task is to analyze the provided video and extract a detailed experimental protocol.
You always stick to the facts in the sources provided, and never make up new facts. 
You use the terminology provided in the glossary accurately throughout your response. 
Take a deep breath and think step by step. 
Answer direct.
"""
prompt_5 = "Input Video:"
prompt_6 = "Output Protocol:"

In [17]:
# Create configuration
config = doc_agent.ProcessingConfig(
    prompts=[prompt_1, prompt_2, prompt_3, prompt_4, prompt_5, prompt_6],
    video_example=video_example,
    file_example=file_example,
    glossary=glossary
)

In [18]:
video_path = "/Users/patriciaskowronek/Documents/LlmProjectIdeas/Documentation/Trimmed_video.mov"

# Process video with caching
# cache.delete()
# output, filename = doc_agent.process_video(video_path, config)

# Process video without caching
output, filename = doc_agent.process_video(video_path, config, use_cache=False)

# Create a markdown file
doc_agent.generate_markdown_for_download(output, filename)

from IPython.display import display, Markdown
print(filename)
Markdown(output)

Uploading file: /Users/patriciaskowronek/Documents/LlmProjectIdeas/Documentation/Trimmed_video.mov
Upload completed. File URI: https://generativelanguage.googleapis.com/v1beta/files/klm9aqxtlacu
.
Video processing completed successfully
prompt_token_count: 57058
candidates_token_count: 1236
total_token_count: 58294

Created markdown file: /var/folders/54/g1_1ycl12hl02xj_g_nm_6cm0000gn/T/Trimmed_video.md
Trimmed_video


# Calibration of a TIMS Device

## Abstract

This protocol describes how to calibrate a trapped ion mobility spectrometry (TIMS) device in timsControl software (version 6.0 or later). It is recommended to calibrate the TIMS device each time before a sample queue is started.


## Materials

### Software

| Software | Version | Purpose | Source/Link |
|----------|----------|----------|------------|
| timsControl | 6.0 (latest) | Software to control timsTof mass spectrometers | Available on every instrument computer |

## Procedure

**Important:** If the instrument has been operated with another source than the CaptiveSpray source, it is highly recommended to wait for 3 hours before initiating the TIMS calibration process. This waiting period permits the temperature and pressure to stabilise, thus ensuring a steady TIMS calibration.

*Estimated timing: 3 minutes*

1. Use timsControl to load a performance evaluation method - a dda-PASEF method. Choose an ion mobility range that matches the ion mobility range intended for use in the study. More details in Figure 4. For proteomics experiments, we typically use an ion mobility range from 0.7 to 1.3 1/K₀, hence the 1/K₀ start and end values can be set to 0.7 and 1.3, respectively. This consistency in ion mobility range allows for the integration of dda-PASEF quality control (QC) runs into the sample table as reference points without necessitating recalibration of the TIMS tunnel.

2. To specifically calibrate a narrow ion mobility range, activate the locked sign at 1/K₀ end (Figure 1: 7). Following this, adjust the 1/K₀ start from 0.7 to 0.85. This alters the ion mobility range from 0.7-1.3 1/K₀ to 0.85-1.45 1/K₀. The aim here is to shift the ion mobility range without modifying the interval, enabling all three calibrants to be used for calibration while maintaining a constant TIMS potential.

3. Wait until the TIC in Chromatogram View is stable. This can take up to 15 minutes. Switch the scan mode to 'MS', set MS averaging to 1 and deactivate the polygon heatmap (Figure 1: 4, 9, 14).

4. In timsControl, navigate to 'calibration', then 'mobility'. From reference lists, select the list '[ESI] Tuning Mix ES-TOF (ESI)' that contains the calibrant masses 622, 922, 1221. Then, specify the linear mode and 5% as detection range and ±0.1 Da as width (Figure 2).

5. Proceed by selecting 'calibrate' (Figure 2). Deselect all calibrants except 622, 922, and 1222 by right-clicking. To verify that the calibrants have been picked correctly at the center, click on them in the reference list. If they are not picked in the center, make adjustments by clicking on the peak in the TIMS view window. If the score is at 100%, press accept.

6. Adjust the ion mobility range to its original values for instance 0.7-1.3 1/K₀, reset the scan mode, set MS averaging to 1, and reactivate the polygon heatmap.


## Expected Results

- The score in the tab "Calibration Mode" should be at 100%


## Troubleshooting

| Problem | Possible Reason | Solution |
|---------|----------------|-----------|
| A weak Tuning Mix calibrant signal | Over time, the calibrant signal of the Tuning Mix may weaken to a point where they are barely visible in the Mobilogram (TIMS View) and in the mass spectrum. | (1) Check if the UltraSource has the gas flow correctly set (Table 1). (2) Cancel the calibration, put the instrument in standby mode, disconnect the filter tubing from the UltraSource (Figure 3), and inject 5 to 10 µL (up to 40 µL) of Tuning Mix directly into the UltraSource. Reattach the filter tubing and switch the instrument into operating mode. Consider increasing the MS averaging value up to a maximum of 30. Repeat this step until a strong calibrant signal is observed and allow a 1-minute pause after each addition of Tuning Mix. The Tuning Mix signal should be more dominant than the signals of common contaminants and reach a signal intensity of around 5x10⁵. If a direct injection of in total 40 µL of Tuning Mix does not significantly enhance the Tuning Mix signal, this may suggest another issue. |


## Figures

**Figure 1: timsControl settings** 
[Placeholder for TimsControl settings image showing the settings interface with labeled parameters 1-16]

**Figure 2: Mass and ion mobility calibration**
[Placeholder for mass and ion mobility calibration screenshots]

**Figure 3: UltraSource**
[Placeholder for UltraSource image showing:
- Front view with labeled components (A-H)
- Back view showing ITI component
- Door view showing components (A, J, H, K)]

## Tables

**Table 1: Gas Flow Parameters for UltraSource**

| Instrument Type | Calibrant 922 Voltage [V] |
|-----------------|-------------------------|
| timsTOF Pro, SCP | 160 |
| timsTOF HT, Ultra | 200 |


# References

1. toDO: Refer to user manual of timsTof
2. toDO: Maybe upload video which shows calibration in action


In [19]:
import gradio as gr
import documentation_agent as doc_agent

css = """
#top-title {font-size: 2rem; text-align: center; margin-bottom: 1rem;}
.download-row {margin-top: 1rem; padding: 1rem; border-top: 1px solid #eee;}
"""

with gr.Blocks(css=css) as iface:
    gr.HTML("""
        <h1 id="top-title">Documentation agent</h1>
    """)
    
    with gr.Row():
        video_input = gr.Video(label="Upload Video")
    
    gr.Markdown("# Protocol extracted from Video:")
    
    with gr.Row():
        text_output = gr.Markdown(label="Output Text")
        filename_state = gr.State()
    
    with gr.Row(elem_classes="download-row"):
        with gr.Column(scale=2):
            download_button = gr.Button("Generate Protocol", variant="primary", size="lg")
        with gr.Column(scale=3):
            file_output = gr.File(label="Download Protocol", file_count="single", type="filepath")

    # Use the existing process_video function directly
    video_input.change(
        fn=lambda x: doc_agent.process_video(x, config),
        inputs=video_input,
        outputs=[text_output, filename_state]
    )

    # Use the existing generate_markdown_for_download function
    download_button.click(
        fn=doc_agent.generate_markdown_for_download,
        inputs=[text_output, filename_state],
        outputs=file_output
    )

iface.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Uploading file: /private/var/folders/54/g1_1ycl12hl02xj_g_nm_6cm0000gn/T/gradio/570716d2985435122e0b104197f76b587c380ff40548f995f2f058999d9079e5/Trimmed_video.mov
Upload completed. File URI: https://generativelanguage.googleapis.com/v1beta/files/nfgy4f0j63c4
.
Video processing completed successfully
prompt_token_count: 57059
candidates_token_count: 2222
total_token_count: 59281
cached_content_token_count: 52335

